## First pratical of the Unsupervised Language Learning course 2017-2018

Joris Mollinga: 11871431 

Aron:

In [36]:
import numpy as np
import sklearn
from sklearn.cluster import KMeans

In [66]:
# Read the word embeddings file
def readWORDSfile(fileName):
    word2vec = {}

    with open (fileName,'r',encoding="utf8") as emb:
        for line in emb:
            row = line.split()
            word = row[0]
            embedding = row[1:]

            # Convert to floats and assign in dictionary
            word2vec[word] = [float(i) for i in embedding] 
    return word2vec

In [67]:
word2vec = readWORDSfile('deps.WORDS')

In [43]:
bow2 = readWORDSfile('bow2.WORDS')

In [ ]:
bow5 = readWORDSfile('bow5.WORDS')